In [1]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
import csv
from tqdm import tqdm
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils as np_utils
import cv2
from matplotlib import pyplot as plt

df = pd.read_csv("aligned_faces_data.csv")


In [2]:
user_id = df['user_id'].tolist()
user_id = list(dict.fromkeys(user_id))



In [3]:
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (227, 227))
            
            
            if img is not None:
                images[filename] = img
    return images

root_folder = 'aligned'
folders = [os.path.join(root_folder, x) for x in user_id]

all_images = {}
for folder in folders:
    files = load_images_from_folder(folder)
    for file in files:
        all_images[file] = files[file]
    


print(len(all_images))



19370


In [4]:


all_images_n = []
for imagename in all_images:
    s = imagename.split('.')
    fname = s[2] +'.'+ s[3]
    face_id = str(s[1])
    all_images_n.append([fname,face_id,all_images[imagename]])
    
print(all_images_n)
    


    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
df2 = pd.DataFrame(all_images_n, columns=['original_image', 'face_id','image_array'])

df2.head()



,original_image,face_id,image_array
0,10287437455_88f52ac14b_o.jpg,2,"[[[126, 142, 113], [117, 133, 104], [122, 138,..."
1,11551644543_3408f718fa_o.jpg,2,"[[[38, 21, 14], [38, 20, 13], [35, 18, 11], [3..."
2,10459364183_1e266cb25b_o.jpg,1,"[[[101, 78, 74], [152, 137, 137], [254, 250, 2..."
3,10152947353_f7fa45a6b5_o.jpg,3,"[[[116, 91, 47], [124, 97, 52], [127, 96, 50],..."
4,9502690171_fa4455d949_o.jpg,7,"[[[153, 125, 103], [153, 127, 104], [160, 134,..."


In [6]:

df['original_image'] = df['original_image'].astype(str)
df2['original_image'] = df2['original_image'].astype(str)
df['face_id'] = df['face_id'].astype(str)
df2['face_id'] = df2['face_id'].astype(str)
mergedf = pd.merge(df, df2, on=['original_image', 'face_id'], how='inner')
mergedf.head(3)




,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score,image_array
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,"(25, 32)",f,0,414,1086,1383,-115,30,17,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,"(25, 32)",m,301,105,640,641,0,0,94,"[[[81, 57, 63], [80, 54, 61], [82, 57, 59], [8..."
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,"(25, 32)",f,2395,876,771,771,175,-30,74,"[[[125, 106, 111], [122, 103, 109], [118, 99, ..."


In [7]:
mergedf.dropna(subset=['gender'], inplace=True)
mergedf = mergedf[mergedf.age != 'None']
mergedf = mergedf[mergedf.gender != 'u']
mergedf = mergedf[mergedf.age != '35']
mergedf = mergedf[mergedf.age != '45']
mergedf = mergedf[mergedf.age != '55']
mergedf = mergedf[mergedf.age != '57']
mergedf = mergedf[mergedf.age != '58']
mergedf = mergedf[mergedf.age != '3']
mergedf = mergedf[mergedf.age != '22']
mergedf = mergedf[mergedf.age != '13']
mergedf = mergedf[mergedf.age != '23']
mergedf = mergedf[mergedf.age != '36']
mergedf = mergedf[mergedf.age != '34']
mergedf = mergedf[mergedf.age != '46']
mergedf = mergedf[mergedf.age != '29']
mergedf = mergedf[mergedf.age != '2']
mergedf = mergedf[mergedf.age != '42']


mergedf.gender[mergedf.gender == 'm'] = 0
mergedf.gender[mergedf.gender == 'f'] = 1


mergedf.head(3)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score,image_array
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,"(25, 32)",1,0,414,1086,1383,-115,30,17,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,"(25, 32)",0,301,105,640,641,0,0,94,"[[[81, 57, 63], [80, 54, 61], [82, 57, 59], [8..."
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,"(25, 32)",1,2395,876,771,771,175,-30,74,"[[[125, 106, 111], [122, 103, 109], [118, 99, ..."


In [8]:
X = mergedf['image_array'].tolist()

y = mergedf['gender'].tolist()




In [9]:
X = np.array(X)
y = np.array(y)


In [10]:
print(X.shape)

(16358, 227, 227, 3)


In [11]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

num_ch_c1 = 50  # Question 2
num_ch_c2 = 60  # Question 2

epochs = 1000  # Fixed
batch_size = 128  # Fixed
learning_rate = 0.001


loss = tf.keras.losses.SparseCategoricalCrossentropy()


optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, test_size=0.33, random_state=42)

In [13]:
model = tf.keras.Sequential()
model.add(layers.Input(shape=(227,227,3)))

model.add(layers.Conv2D(num_ch_c1, (9,9), activation='relu', input_shape=(None, None, 3), padding='VALID'))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='VALID'))
model.add(layers.Conv2D(num_ch_c2, (5,5), activation='relu', padding='VALID'))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='VALID'))
model.add(layers.Flatten())
model.add(layers.Dense(300))
model.add(layers.Dense(10, input_shape=(300,), activation='softmax')) # Here no softmax because we have combined it with the loss



In [14]:
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, 
                         y_test))




Epoch 1/1000
 5/86 [>.............................] - ETA: 17:42 - loss: 5607.2617 - accuracy: 0.4109

KeyboardInterrupt: 

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

train_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']

plt.plot(range(1, len(train_acc) + 1), train_acc, label='Train')
plt.plot(range(1, len(test_acc) + 1), test_acc, label='Test')
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.show()

plt.savefig('accuracy.pdf')